In [ ]:
import numpy as np 
import pandas as pd
from konlpy.tag import Okt

callcount = 0
def to_noun_sentence(text):
    global callcount
    callcount += 1
    if callcount % 100 == 0:
        print(callcount)
    stopwords = ['질문', '문의', '관련', '그대로', '계속', '답변', '선생님', '관련문의',
            '한지', '자주', '좀', '쪽', '자꾸', '요즘', '몇개', '무조건', '하나요',
            '안해','요', '경우', '최근', '및', '몇', '달', '일반', '전날', '저번',
            '말', '일어나지', '며칠', '먹기', '지난번', '글', '때문', '너', '무',
            '오늘', '시', '잔', '뒤', '지속', '막', '것', '이건', '뭔가', '다시', '그',
                '무슨', '안', '난', '도', '기', '후', '거리', '이', '뭘', '저', '뭐', '답젼',
                '평생', '회복', '반', '감사', '의사', '보험', '학생', '제발', '살짝',
                '느낌', '제', '대해','갑자기','문제', '전','정도', '왜', '거', '가요',
                '의심', '어제', '추천', '를', '지금', '무엇', '내일', '관해', '리', '세',
                 '로', '목적', '그냥', '거의', '고민', '다음', '이틀', '항상', '뭐', '때',
                '요', '가끔', '이후', '혹시', ]
    twitter = Okt()
    nouns = twitter.nouns(text)
    for word in nouns:
        if word in stopwords:
            while word in nouns:
                nouns.remove(word)
    return ' '.join(nouns)

def preprocess(filepath, exportpath, startindex=0, endindex=None):
    df = pd.read_csv(filepath)
    if endindex != None:
        df = df[startindex:endindex]
    
    df.dropna(inplace=True)
    df['symptom'] = df['symptom'].str.replace(pat='([ㄱ-ㅎㅏ-ㅣ]+)', repl=r' ', regex=True)
    df['symptom'] = df['symptom'].str.replace(pat='[^\w\s]', repl=r' ', regex=True)
    
    df['nouns'] = df['symptom'].apply(to_noun_sentence)
    
    #라벨 정수화
    class_to_label = {'DERM': 0, 'GS': 1, 'IP': 2, 'GI':3, 'OPH':4,
                  'NR': 5, 'ENT': 6, 'PSY': 7, 'HEON': 8, 'RHEU': 9,
                  'REHM': 10, 'NS': 11, 'AN': 12, 'DENT': 13, 'PS': 14,
                  'CS': 15, 'INFC': 16, 'OS': 17,
                  'EMR': 18, 'ENDO': 19, 'CA': 20, 'KTM': 21, 'OBGY': 22,
                  'URO': 23, 'ALL': 24, 'NPH': 25,'LAB': 26}
    df['label'] = df['class'].map(class_to_label)
    
    # 2개 컬럼만 남김
    df = df[['nouns', 'label']]
    
    # NaN 삭제
    df.dropna(inplace=True)
    df['label'] = df['label'].astype(int)
    
    df.to_csv(exportpath, encoding='utf-8', index=False)
    print('done')